In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# ScipyKrylov

ScipyKrylov is an iterative linear solver that wraps the methods found in `scipy.sparse.linalg`.
The default method is "gmres", or the Generalized Minimal RESidual method. Support for other
`scipy.sparse.linalg` solvers will be added over time. This linear solver is capable of handling any
system topology very effectively. It also solves all subsystems below it in the hierarchy, so
assigning different solvers to subsystems will have no effect on the solution at this level.

This is a serial solver, so it should never be used under MPI; use [PETScKrylov](../../../_srcdocs/packages/solvers.linear/petsc_ksp) instead.

Here, we calculate the total derivatives across the Sellar system.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src41", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src41`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src42", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src42`
:::

In [ ]:
import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.ScipyKrylov()

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(J['obj', 'z'][0][0], 9.61001056, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448534, .00001)

## ScipyKrylov Options

In [ ]:
om.show_options_table("openmdao.solvers.linear.scipy_iter_solver.ScipyKrylov")

## ScipyKrylov Constructor

The call signature for the `ScipyKrylov` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linear.scipy_iter_solver.ScipyKrylov.__init__
        :noindex:
```

## ScipyKrylov Option Examples

**maxiter**

  `maxiter` lets you specify the maximum number of GMRES iterations to apply. The default maximum is 1000, which
  is much higher than the other linear solvers because each multiplication by the system Jacobian is considered
  to be an iteration. You may have to decrease this value if you have a coupled system that is converging
  very slowly. (Of course, in such a case, it may be better to add a preconditioner.)  Alternatively, you
  may have to raise it if you have an extremely large number of components in your system (a 1000-component
  ring would need 1000 iterations just to make it around once.)

  This example shows what happens if you set `maxiter` too low. Since iteration is terminated after only 3 iterations, the solver does not converge and the derivatives will not be correct.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.ScipyKrylov()
model.linear_solver.options['maxiter'] = 3

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

Depending on your situation, you may want to set the `err_on_non_converge` option to `True`, so that this will raise an AnalysisError.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives
from openmdao.api import AnalysisError

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.ScipyKrylov()
model.linear_solver.options['maxiter'] = 3
model.linear_solver.options['err_on_non_converge'] = True

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

try:
    J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')
except AnalysisError as err:
    # Special handling for a failure to converge when computing derivatives
    print("===\nLinear Solver failed to converge, derivatives are not valid.\n===")

**atol**

  Here, we set the absolute tolerance to a much tighter value (default is 1.0e-12) to show what happens. In
  practice, the tolerance serves a dual role in GMRES. In addition to being a termination criteria, the tolerance
  also defines what GMRES considers to be tiny. Tiny numbers are replaced by zero when the argument vector is
  normalized at the start of each new matrix-vector product. The end result here is that we iterate longer to get
  a marginally better answer.

  You may need to adjust this setting if you have abnormally large or small values in your global Jacobian.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.nonlinear_solver = om.NonlinearBlockGS()

model.linear_solver = om.ScipyKrylov()
model.linear_solver.options['atol'] = 1.0e-20

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

wrt = ['z']
of = ['obj']

J = prob.compute_totals(of=of, wrt=wrt, return_format='flat_dict')

In [ ]:
print(J['obj', 'z'][0][0])
print(J['obj', 'z'][0][1])

In [ ]:
assert_near_equal(J['obj', 'z'][0][0], 9.61001055699, .00001)
assert_near_equal(J['obj', 'z'][0][1], 1.78448533563, .00001)

**rtol**

  The 'rtol' setting is not supported by SciPy GMRES for versions earlier than SciPy 1.12.0.

## Specifying a Preconditioner

You can specify a preconditioner to improve the convergence of the iterative linear solution by setting the `precon` attribute. The
motivation for using a preconditioner is the observation that iterative methods have better convergence
properties if the linear system has a smaller condition number, so the goal of the preconditioner is to
improve the condition number in part or all of the Jacobian.

Here, we add a Gauss-Seidel preconditioner to a problem that contains two subgroups, each with an implicit component that implements a quadratic
equation. These are solved together by a Newton solver at the top.  The goal of the preconditioner here is to solve the smaller linear systems
for the quadratic components independently, and use that solution to precondition the full system solution. This is accomplished by setting up
the linear solver hierarchy so that the preconditioner is `LinearBlockGS` and the subsytems `sub1` and `sub2` contain a `DirectSolver`.

Note that the number of GMRES iterations is lower when using the preconditioner.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src43", get_code("openmdao.test_suite.components.quad_implicit.QuadraticComp"), display=False)

:::{Admonition} `QuadraticComp` class definition 
:class: dropdown

{glue:}`code_src43`
:::

In [ ]:
from openmdao.test_suite.components.quad_implicit import QuadraticComp

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group())
sub1.add_subsystem('q1', QuadraticComp())
sub1.add_subsystem('z1', om.ExecComp('y = -6.0 + .01 * x'))
sub2 = model.add_subsystem('sub2', om.Group())
sub2.add_subsystem('q2', QuadraticComp())
sub2.add_subsystem('z2', om.ExecComp('y = -6.0 + .01 * x'))

model.connect('sub1.q1.x', 'sub1.z1.x')
model.connect('sub1.z1.y', 'sub2.q2.c')
model.connect('sub2.q2.x', 'sub2.z2.x')
model.connect('sub2.z2.y', 'sub1.q1.c')

model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.ScipyKrylov()

prob.setup()

model.sub1.linear_solver = om.DirectSolver()
model.sub2.linear_solver = om.DirectSolver()

model.linear_solver.precon = om.LinearBlockGS()

prob.set_solver_print(level=2)
prob.run_model()

In [ ]:
print(prob.get_val('sub1.q1.x'))
print(prob.get_val('sub2.q2.x'))

In [ ]:
assert_near_equal(prob.get_val('sub1.q1.x'), 1.996, .0001)
assert_near_equal(prob.get_val('sub2.q2.x'), 1.996, .0001)